# NASM ML Models - Model Registry

This notebook trains ML models for the NASM Intelligence Agent:
- **Exam Success Prediction** - Predict likelihood of passing certification exam
- **Enrollment Demand Forecast** - Predict future course enrollment rates
- **Student Churn Prediction** - Predict students at risk of not recertifying

All models are registered to Snowflake Model Registry and can be added as tools to the Intelligence Agent.

## Prerequisites

**Required Packages** (configured automatically):
- `snowflake-ml-python`
- `scikit-learn`
- `xgboost`
- `matplotlib`

**Database Context:**
- **Database:** NASM_INTELLIGENCE  
- **Schema:** ANALYTICS  
- **Warehouse:** NASM_WH

**Note:** This notebook uses Snowflake Model Registry. Ensure you have appropriate permissions to create and register models.


In [ ]:
# Import Python packages
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Import Snowpark
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark import Window

# Import Snowpark ML
from snowflake.ml.modeling.preprocessing import StandardScaler, OneHotEncoder
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.linear_model import LinearRegression, LogisticRegression
from snowflake.ml.modeling.ensemble import RandomForestClassifier
from snowflake.ml.modeling.metrics import mean_squared_error, mean_absolute_error, accuracy_score, roc_auc_score
from snowflake.ml.registry import Registry

print("✅ Packages imported successfully")


In [ ]:
# Get active Snowflake session
session = get_active_session()

# Set context - MUST MATCH 01_database_and_schema.sql
session.use_database('NASM_INTELLIGENCE')
session.use_schema('ANALYTICS')
session.use_warehouse('NASM_WH')

print(f"✅ Connected - Role: {session.get_current_role()}")
print(f"   Warehouse: {session.get_current_warehouse()}")
print(f"   Database.Schema: {session.get_fully_qualified_current_schema()}")


## MODEL 1: Exam Success Prediction

Predict whether a student will pass their certification exam based on study behavior and profile features.


In [ ]:
# Get exam success data with features
# COLUMN NAMES VERIFIED against 02_create_tables.sql
exam_success_df = session.sql("""
SELECT
    ex.exam_id,
    s.years_in_fitness::FLOAT AS years_experience,
    CASE WHEN s.education_level IN ('Bachelors Degree', 'Masters Degree', 'Doctorate') THEN 1 ELSE 0 END AS has_degree,
    e.study_progress_pct::FLOAT AS study_progress,
    e.total_study_hours::FLOAT AS study_hours,
    e.modules_completed::FLOAT AS modules_done,
    ct.certification_category AS cert_category,
    s.student_type AS student_type,
    ex.attempt_number::FLOAT AS attempt_num,
    CASE WHEN ex.passed = TRUE THEN 1 ELSE 0 END AS passed_exam
FROM RAW.EXAMS ex
JOIN RAW.STUDENTS s ON ex.student_id = s.student_id
JOIN RAW.ENROLLMENTS e ON ex.enrollment_id = e.enrollment_id
JOIN RAW.CERTIFICATION_TYPES ct ON ex.certification_type_id = ct.certification_type_id
WHERE ex.exam_status = 'COMPLETED'
  AND ex.exam_date >= DATEADD('year', -2, CURRENT_DATE())
LIMIT 15000
""")

print(f"Exam success data: {exam_success_df.count()} exam records")
exam_success_df.show(5)


In [ ]:
# Train/test split (80/20)
train_exam, test_exam = exam_success_df.random_split([0.8, 0.2], seed=42)
train_exam = train_exam.drop("EXAM_ID")
test_exam = test_exam.drop("EXAM_ID")

# Create pipeline with preprocessing and classification
exam_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["CERT_CATEGORY", "STUDENT_TYPE"],
        output_cols=["CERT_CATEGORY_ENC", "STUDENT_TYPE_ENC"],
        drop_input_cols=True,
        handle_unknown="ignore"
    )),
    ("Scaler", StandardScaler(
        input_cols=["YEARS_EXPERIENCE", "STUDY_PROGRESS", "STUDY_HOURS", "MODULES_DONE", "ATTEMPT_NUM"],
        output_cols=["YEARS_EXPERIENCE_SCALED", "STUDY_PROGRESS_SCALED", "STUDY_HOURS_SCALED", "MODULES_DONE_SCALED", "ATTEMPT_NUM_SCALED"]
    )),
    ("Classifier", RandomForestClassifier(
        label_cols=["PASSED_EXAM"],
        output_cols=["PREDICTED_PASS"],
        n_estimators=100,
        max_depth=10
    ))
])

# Train model
exam_pipeline.fit(train_exam)
print("✅ Exam success prediction model trained")


In [ ]:
# Evaluate and register exam success model
exam_predictions = exam_pipeline.predict(test_exam)

accuracy = accuracy_score(
    df=exam_predictions, 
    y_true_col_names="PASSED_EXAM", 
    y_pred_col_names="PREDICTED_PASS"
)

exam_metrics = {"accuracy": round(accuracy, 4)}
print(f"Model metrics: {exam_metrics}")

# Register model
reg = Registry(session)
reg.log_model(
    model=exam_pipeline,
    model_name="EXAM_SUCCESS_PREDICTOR",
    version_name="V1",
    comment="Predicts likelihood of passing certification exam using Random Forest based on study behavior and student profile",
    metrics=exam_metrics
)

print("✅ Exam success model registered to Model Registry as EXAM_SUCCESS_PREDICTOR")


## MODEL 2: Enrollment Demand Forecaster

Predict future enrollment demand based on historical monthly patterns.

**CRITICAL:** Column names and data types MUST match `07_create_model_wrapper_functions.sql` EXACTLY:
- `MONTH_NUM` - INTEGER (no cast to float!)
- `YEAR_NUM` - INTEGER (no cast to float!)
- `UNIQUE_STUDENTS` - FLOAT
- `AVG_ENROLLMENT_PRICE` - FLOAT
- `TOTAL_REVENUE` - FLOAT
- `ENROLLMENT_COUNT` - FLOAT (label column)
- Output: `PREDICTED_ENROLLMENTS`


In [ ]:
# Get monthly enrollment data for forecasting
# COLUMN NAMES AND TYPES VERIFIED against 07_create_model_wrapper_functions.sql lines 136-148
# CRITICAL: MONTH_NUM and YEAR_NUM are INTEGER (no ::FLOAT cast!)
enrollment_forecast_df = session.sql("""
SELECT
    MONTH(e.enrollment_date) AS month_num,
    YEAR(e.enrollment_date) AS year_num,
    COUNT(DISTINCT e.student_id)::FLOAT AS unique_students,
    AVG(e.net_price)::FLOAT AS avg_enrollment_price,
    SUM(e.net_price)::FLOAT AS total_revenue,
    COUNT(DISTINCT e.enrollment_id)::FLOAT AS enrollment_count
FROM RAW.ENROLLMENTS e
WHERE e.enrollment_date >= DATEADD('year', -3, CURRENT_DATE())
  AND e.enrollment_status IN ('COMPLETED', 'ACTIVE')
GROUP BY MONTH(e.enrollment_date), YEAR(e.enrollment_date)
ORDER BY year_num, month_num
""")

print(f"Enrollment forecast data: {enrollment_forecast_df.count()} monthly records")
enrollment_forecast_df.show(10)


In [ ]:
# Train/test split (80/20)
train_enroll, test_enroll = enrollment_forecast_df.random_split([0.8, 0.2], seed=42)

# Create pipeline for enrollment forecasting
# Using LinearRegression since this is a regression problem (predicting counts)
enrollment_pipeline = Pipeline([
    ("Scaler", StandardScaler(
        input_cols=["UNIQUE_STUDENTS", "AVG_ENROLLMENT_PRICE", "TOTAL_REVENUE"],
        output_cols=["UNIQUE_STUDENTS_SCALED", "AVG_ENROLLMENT_PRICE_SCALED", "TOTAL_REVENUE_SCALED"]
    )),
    ("Regressor", LinearRegression(
        input_cols=["MONTH_NUM", "YEAR_NUM", "UNIQUE_STUDENTS_SCALED", "AVG_ENROLLMENT_PRICE_SCALED", "TOTAL_REVENUE_SCALED"],
        label_cols=["ENROLLMENT_COUNT"],
        output_cols=["PREDICTED_ENROLLMENTS"]
    ))
])

# Train model
enrollment_pipeline.fit(train_enroll)
print("✅ Enrollment demand forecaster model trained")


In [ ]:
# Evaluate and register enrollment forecaster model
enrollment_predictions = enrollment_pipeline.predict(test_enroll)

mse = mean_squared_error(
    df=enrollment_predictions,
    y_true_col_names="ENROLLMENT_COUNT",
    y_pred_col_names="PREDICTED_ENROLLMENTS"
)
mae = mean_absolute_error(
    df=enrollment_predictions,
    y_true_col_names="ENROLLMENT_COUNT",
    y_pred_col_names="PREDICTED_ENROLLMENTS"
)

enrollment_metrics = {"mse": round(mse, 2), "mae": round(mae, 2)}
print(f"Model metrics: {enrollment_metrics}")

# Register model
reg.log_model(
    model=enrollment_pipeline,
    model_name="ENROLLMENT_DEMAND_FORECASTER",
    version_name="V1",
    comment="Forecasts monthly enrollment demand using Linear Regression based on historical patterns",
    metrics=enrollment_metrics
)

print("✅ Enrollment forecaster model registered to Model Registry as ENROLLMENT_DEMAND_FORECASTER")


## MODEL 3: Student Churn Predictor

Predict which students are at risk of not recertifying based on engagement patterns.

**CRITICAL:** Column names and data types MUST match `07_create_model_wrapper_functions.sql` EXACTLY (lines 228-240):
- `DAYS_INACTIVE` - FLOAT
- `LIFETIME_SPEND` - FLOAT
- `COURSES_COMPLETED` - FLOAT
- `TOTAL_CEUS` - FLOAT
- `CURRENT_CEUS` - FLOAT
- `REQUIRED_CEUS` - FLOAT
- `CEU_COMPLETION_PCT` - FLOAT
- `DAYS_TO_EXPIRY` - FLOAT
- `PAST_RENEWALS` - FLOAT
- `CHURNED` - INTEGER (0 or 1, label column)
- Output: `PREDICTED_CHURN`


In [ ]:
# Get student churn data with engagement features
# COLUMN NAMES AND TYPES VERIFIED against 07_create_model_wrapper_functions.sql lines 228-240
# COLUMN SOURCES VERIFIED against 02_create_tables.sql:
#   - STUDENTS: last_activity_date (line 62), total_spend (line 65), total_courses_completed (line 63), lifetime_ceus_earned (line 66)
#   - CERTIFICATIONS: certification_status (line 141), ceus_earned (line 142), ceus_required (line 143), expiry_date (line 140), renewal_count (line 145)
churn_df = session.sql("""
SELECT
    c.certification_id,
    DATEDIFF('day', s.last_activity_date, CURRENT_DATE())::FLOAT AS days_inactive,
    s.total_spend::FLOAT AS lifetime_spend,
    s.total_courses_completed::FLOAT AS courses_completed,
    s.lifetime_ceus_earned::FLOAT AS total_ceus,
    c.ceus_earned::FLOAT AS current_ceus,
    c.ceus_required::FLOAT AS required_ceus,
    COALESCE((c.ceus_earned::FLOAT / NULLIF(c.ceus_required, 0) * 100), 0)::FLOAT AS ceu_completion_pct,
    DATEDIFF('day', CURRENT_DATE(), c.expiry_date)::FLOAT AS days_to_expiry,
    c.renewal_count::FLOAT AS past_renewals,
    -- Create synthetic churn label based on engagement patterns
    -- Students with low engagement and approaching expiry are more likely to churn
    CASE 
        WHEN c.certification_status = 'EXPIRED' THEN 1
        WHEN c.certification_status = 'LAPSED' THEN 1
        WHEN DATEDIFF('day', s.last_activity_date, CURRENT_DATE()) > 180 
             AND (c.ceus_earned::FLOAT / NULLIF(c.ceus_required, 0)) < 0.5 THEN 1
        ELSE 0
    END AS churned
FROM RAW.CERTIFICATIONS c
JOIN RAW.STUDENTS s ON c.student_id = s.student_id
WHERE c.expiry_date IS NOT NULL
LIMIT 15000
""")

print(f"Churn prediction data: {churn_df.count()} certification records")
churn_df.show(5)


In [ ]:
# Train/test split (80/20)
train_churn, test_churn = churn_df.random_split([0.8, 0.2], seed=42)
train_churn = train_churn.drop("CERTIFICATION_ID")
test_churn = test_churn.drop("CERTIFICATION_ID")

# Create pipeline for churn prediction
# Using RandomForestClassifier since this is a binary classification problem
churn_pipeline = Pipeline([
    ("Scaler", StandardScaler(
        input_cols=["DAYS_INACTIVE", "LIFETIME_SPEND", "COURSES_COMPLETED", "TOTAL_CEUS", 
                    "CURRENT_CEUS", "REQUIRED_CEUS", "CEU_COMPLETION_PCT", "DAYS_TO_EXPIRY", "PAST_RENEWALS"],
        output_cols=["DAYS_INACTIVE_SCALED", "LIFETIME_SPEND_SCALED", "COURSES_COMPLETED_SCALED", "TOTAL_CEUS_SCALED",
                     "CURRENT_CEUS_SCALED", "REQUIRED_CEUS_SCALED", "CEU_COMPLETION_PCT_SCALED", "DAYS_TO_EXPIRY_SCALED", "PAST_RENEWALS_SCALED"]
    )),
    ("Classifier", RandomForestClassifier(
        label_cols=["CHURNED"],
        output_cols=["PREDICTED_CHURN"],
        n_estimators=100,
        max_depth=10
    ))
])

# Train model
churn_pipeline.fit(train_churn)
print("✅ Student churn prediction model trained")


In [ ]:
# Evaluate and register churn prediction model
churn_predictions = churn_pipeline.predict(test_churn)

accuracy = accuracy_score(
    df=churn_predictions,
    y_true_col_names="CHURNED",
    y_pred_col_names="PREDICTED_CHURN"
)

churn_metrics = {"accuracy": round(accuracy, 4)}
print(f"Model metrics: {churn_metrics}")

# Register model
reg.log_model(
    model=churn_pipeline,
    model_name="STUDENT_CHURN_PREDICTOR",
    version_name="V1",
    comment="Predicts likelihood of student not recertifying using Random Forest based on engagement patterns",
    metrics=churn_metrics
)

print("✅ Churn prediction model registered to Model Registry as STUDENT_CHURN_PREDICTOR")


## Summary

All 3 models have been trained and registered to Snowflake Model Registry:

| Model | Type | Output Column | Purpose |
|-------|------|---------------|---------|
| `EXAM_SUCCESS_PREDICTOR` | RandomForestClassifier | `PREDICTED_PASS` | Predicts exam pass likelihood |
| `ENROLLMENT_DEMAND_FORECASTER` | LinearRegression | `PREDICTED_ENROLLMENTS` | Forecasts monthly enrollments |
| `STUDENT_CHURN_PREDICTOR` | RandomForestClassifier | `PREDICTED_CHURN` | Identifies churn risk |

**Next Steps:**
1. Run `sql/ml/07_create_model_wrapper_functions.sql` to create stored procedures that wrap these models
2. Run `sql/agent/08_create_intelligence_agent.sql` to create the Intelligence Agent with ML tools
3. Test the agent in Snowsight under AI & ML > Agents


In [ ]:
# Verify all models are registered
print("=" * 60)
print("VERIFICATION: Models registered in Model Registry")
print("=" * 60)

# List all models in registry
models = reg.show_models()
models.show()

print("\n✅ All 3 models registered successfully!")
print("   - EXAM_SUCCESS_PREDICTOR")
print("   - ENROLLMENT_DEMAND_FORECASTER")
print("   - STUDENT_CHURN_PREDICTOR")
